<a href="https://colab.research.google.com/github/Lkrasnop/Master-degree-Data_science_final_project/blob/data-preparation/Final_Project_EDA_01_10_2023_class_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final project - Lior Krasnopolski and Karin Shauli - version 01.10.2023

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# פילטרים כללים:

בין הגילאים 20 -80

לעבוד רק על האוכלסייה הגברית

# סיכום דאטא פריימים:

1.**df_info** - data frame of general info for each patient.

2.**df_GSE87571_normalized_celltype** - normalized data for each probe per patient.

3.**df_gene_data** - dataframe of info (such as gene name) for each cg(probe).

4.**df_GSE87571_normalized_celltype_reconstruct** - reconstruct the dataframe that the cg would be column and not header.

5.**df_GSE87571_normalized_celltype_and_patient_info** - merged dataframe between *df_info, df_gene_data, df_GSE87571_normalized_celltype_reconstruct*

6.**df_GSE87571_final_group_by_gender** - group   df_GSE87571_normalized_celltype_and_patient_info by gender, age

# משימות לתאריך 02.10.2023:

1. איחוד בין רשימת הגנים במאמר לבין רשימת הגנים הקיימים אצלנו.

2. להדפיס קורולציה מסוג קלאסטר רק עבור הגנים שהייתה להם התאמה בסעיף ראשון

3. לבחור שתיים שלוש גנים מתוך סעיף 2 לפי לוגיקה מסויימת.

4. להדפיס גרף של שונות כתלות בגיל (כל חמש שנים) עבור גנים מסעיף שלוש .

5. להדפיס גרף של ממוצע (נראה לי שזה מה שנאמר בישיבה) כתלות בגיל עבור הגנים מסעיף שלוש.



In [ ]:
import pandas as pd
from statistics import mean
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.cluster as cluster
from sklearn.cluster import KMeans
import numpy as np

In [ ]:
class data_preparation:
  def __init__(self
               # parameters of our source
               ,our_path_gene_source
               ,number_of_splits
               ,gene_primry_desc
               ,elias_gene_desc
               ,symbol_splitting
               #article parameters
               ,article_path_gen_source
               ,article_gene_primry_desc
               ,article_gene_elias_desc
               ,article_symbol_splitting):

    self.our_path_gene_source = our_path_gene_source
    self.number_of_splits = number_of_splits
    self.gene_primry_desc = gene_primry_desc
    self.elias_gene_desc = elias_gene_desc
    self.symbol_splitting = symbol_splitting
    self.article_gene_primry_desc = article_gene_primry_desc
    self.symbol_splitting_article = article_symbol_splitting
    self.article_path_gen_source = article_path_gen_source
    self.article_gene_elias_desc = article_gene_elias_desc

    #perapre our gene data set
  def our_gene_source(self):
      numer_of_splits = self.number_of_splits
      df_our_gene_data = pd.read_csv('{}'.format(self.our_path_gene_source))
      df_our_gene_data = df_our_gene_data[['{}'.format(self.gene_primry_desc),'{}'.format(self.elias_gene_desc)]].drop_duplicates().reset_index(drop = True)

      # usage of refgene name and spliting by ";" symbol
      df_our_gene_data = df_our_gene_data['{}'.format(self.elias_gene_desc)].str.split('{}'.format(self.symbol_splitting), n=numer_of_splits, expand=True)

      for elem in range(0,len(df_our_gene_data.columns)):
        df_our_gene_data.rename(columns={elem:"alias-our-dataframe {}".format(elem+1)},inplace =True)

      return  df_our_gene_data

  def article_gene_source(self):
    # gert of list_sheet_names
    df = pd.ExcelFile('{}'.format(self.article_path_gen_source))
    list_sheet_names = df.sheet_names
    del df

    # get dataframe for gene dataset from aritcle
    data_frame_gene = []

    for name in list_sheet_names:
      temp = pd.read_excel('{}'.format(self.article_path_gen_source) , sheet_name='{}'.format(name))
      temp = temp[['{}'.format(self.article_gene_primry_desc),'{}'.format(self.article_gene_elias_desc)]]
      temp['group_name'] = name
      data_frame_gene.append(temp)

    data_frame_gene = pd.concat(data_frame_gene)

    #split the alias
    numer_of_splits = self.number_of_splits
    split_alias = data_frame_gene['{}'.format(self.article_gene_elias_desc)].str.split('{}'.format(self.symbol_splitting_article), n=numer_of_splits, expand=True)

    #change names of added columns
    for elem in range(0,len(split_alias.columns)):
      split_alias.rename(columns={elem:"alias {}".format(elem+1)},inplace =True)

    #merge alias splited dataframe into gene frame
    data_frame_gene = pd.concat([data_frame_gene, split_alias], axis=1)

    #drop alias inital column
    data_frame_gene = data_frame_gene.drop(['{}'.format(self.article_gene_elias_desc)], axis = 1)

    return data_frame_gene


In [ ]:
#article parameters
path = '/content/drive/MyDrive/Final Project Data Science Master/קבצים ישנים/all_450K.csv'
number_of_splits = 13
gene_primry_desc = 'Name'
elias_gene_desc = 'UCSC_RefGene_Name'
symbol_splitting = ';'

#article parameters
article_path_gen_source = '/content/drive/MyDrive/Final Project Data Science Master/ 41419_2021_4121_MOESM1_ESM_cleaned.xlsx'
article_gene_primry_desc = 'Official Gene symbol'
article_gene_elias_desc = 'Alias (optional)'
symbol_splitting_article = ','



data_preparation = data_preparation(path,
                                    number_of_splits,
                                    gene_primry_desc,
                                    elias_gene_desc,
                                    symbol_splitting,
                                    article_path_gen_source,
                                    article_gene_primry_desc,
                                    article_gene_elias_desc,
                                    symbol_splitting_article)

In [ ]:
our_data = data_preparation.our_gene_source()

<ipython-input-24-2b2fe1bb1f5f>:29: DtypeWarning: Columns (2,4,11,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_our_gene_data = pd.read_csv('{}'.format(self.our_path_gene_source))


In [ ]:
our_data['alias-our-dataframe 1'].nunique()

20634

In [ ]:
articles_data = data_preparation.article_gene_source()
articles_data

,Official Gene symbol,group_name,alias 1,alias 2,alias 3,alias 4,alias 5,alias 6,alias 7,alias 8,alias 9,alias 10,alias 11,alias 12,alias 13
0,LAMTOR1,mTOR and upstream pathways,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LAMTOR2,mTOR and upstream pathways,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LAMTOR3,mTOR and upstream pathways,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LAMTOR4,mTOR and upstream pathways,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LAMTOR5,mTOR and upstream pathways,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,RAB34,Lysosome-related genes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,RILPL1,Lysosome-related genes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,SCPEP1,Lysosome-related genes,HSCP1,RISC,None,None,None,None,None,None,None,None,None,None,None
32,SLC38A9,Lysosome-related genes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
articles_data['Official Gene symbol'].nunique()

# import normalized celltype dataframe

In [ ]:
df_GSE87571_normalized_celltype_reconstruct  = pd.read_csv('/content/drive/MyDrive/Final Project Data Science Master/df_GSE87571_normalized_celltype_reconstruct.csv')
df_GSE87571_normalized_celltype_reconstruct = df_GSE87571_normalized_celltype_reconstruct.drop(columns = ['Unnamed: 0'] , axis=1)
df_GSE87571_normalized_celltype_reconstruct

,source_name,cg,normalized
0,GSM2333902,cg00000108,0.991673
1,GSM2333903,cg00000108,0.993019
2,GSM2333901,cg00000108,0.992090
3,GSM2333905,cg00000108,0.993610
4,GSM2333908,cg00000108,0.987779
...,...,...,...
97070767,GSM2334627,cg27666123,0.963326
97070768,GSM2334630,cg27666123,0.968636
97070769,GSM2334631,cg27666123,0.944395
97070770,GSM2334632,cg27666123,0.933849


# taking only the relevant cases : filtering only the relevant probes and patients.

In [ ]:
# drop all irrelevant patients and cgs
df_GSE87571_normalized_celltype_reconstruct = df_GSE87571_normalized_celltype_reconstruct.loc[df_GSE87571_normalized_celltype_reconstruct.source_name.isin(df_info.source_name)]
df_GSE87571_normalized_celltype_reconstruct = df_GSE87571_normalized_celltype_reconstruct.loc[df_GSE87571_normalized_celltype_reconstruct.cg.isin(df_our_gene_data.Name)]
df_GSE87571_normalized_celltype_reconstruct = df_GSE87571_normalized_celltype_reconstruct.dropna()
display(df_GSE87571_normalized_celltype_reconstruct)

,source_name,cg,normalized
831,GSM2333902,cg00000321,0.267138
832,GSM2333903,cg00000321,0.286995
833,GSM2333901,cg00000321,0.329600
834,GSM2333905,cg00000321,0.318397
835,GSM2333908,cg00000321,0.260389
...,...,...,...
97070490,GSM2334627,cg27666046,0.553640
97070491,GSM2334630,cg27666046,0.606716
97070492,GSM2334631,cg27666046,0.543427
97070493,GSM2334632,cg27666046,0.596561


# merged between normalized (probes) dataframe and pateints' general info


In [ ]:
#merge between normalized dataframe and the patients'info
df_GSE87571_normalized_celltype_and_patient_info= pd.merge(df_GSE87571_normalized_celltype_reconstruct,
                     df_info,
                     on=['source_name'],
                     how='left')
del df_GSE87571_normalized_celltype_reconstruct , df_info

In [ ]:
#merge between normalized dataframe and the genes' info
df_GSE87571_normalized_celltype_and_patient_info= pd.merge(df_GSE87571_normalized_celltype_and_patient_info,
                     df_our_gene_data,
                     right_on=['Name'],
                     left_on=['cg'],
                     how='left')
del df_our_gene_data

### drop irrelevant columns

In [ ]:
df_GSE87571_normalized_celltype_and_patient_info = df_GSE87571_normalized_celltype_and_patient_info[['cg','normalized','age','UCSC_RefGene_Name']]
df_GSE87571_normalized_celltype_and_patient_info.rename(columns={'UCSC_RefGene_Name':"gene"},inplace =True)
display(df_GSE87571_normalized_celltype_and_patient_info)

<ipython-input-24-ebd69ee66860>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_GSE87571_normalized_celltype_and_patient_info.rename(columns={'UCSC_RefGene_Name':"gene"},inplace =True)


,cg,normalized,age,gene
0,cg00000321,0.267138,55.0,SFRP1
1,cg00000321,0.286995,23.0,SFRP1
2,cg00000321,0.329600,72.0,SFRP1
3,cg00000321,0.318397,74.0,SFRP1
4,cg00000321,0.260389,38.0,SFRP1
...,...,...,...,...
63296157,cg27666046,0.553640,53.0,SECTM1
63296158,cg27666046,0.606716,70.0,SECTM1
63296159,cg27666046,0.543427,72.0,SECTM1
63296160,cg27666046,0.596561,54.0,SECTM1


# saving the final dataframe

In [ ]:
df_GSE87571_normalized_celltype_and_patient_info = pd.read_csv('/content/drive/MyDrive/Final Project Data Science Master/df_final.csv')
df_GSE87571_normalized_celltype_and_patient_info = df_GSE87571_normalized_celltype_and_patient_info[['cg','normalized','age','UCSC_RefGene_Name']]
df_GSE87571_normalized_celltype_and_patient_info.rename(columns={'UCSC_RefGene_Name':"gene"},inplace =True)
display(df_GSE87571_normalized_celltype_and_patient_info)

<ipython-input-3-cd9f96fd06a4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_GSE87571_normalized_celltype_and_patient_info.rename(columns={'UCSC_RefGene_Name':"gene"},inplace =True)


,cg,normalized,age,gene
0,cg00000321,0.267138,55.0,SFRP1
1,cg00000321,0.286995,23.0,SFRP1
2,cg00000321,0.329600,72.0,SFRP1
3,cg00000321,0.318397,74.0,SFRP1
4,cg00000321,0.260389,38.0,SFRP1
...,...,...,...,...
63296157,cg27666046,0.553640,53.0,SECTM1
63296158,cg27666046,0.606716,70.0,SECTM1
63296159,cg27666046,0.543427,72.0,SECTM1
63296160,cg27666046,0.596561,54.0,SECTM1


In [ ]:
df_GSE87571_normalized_celltype_and_patient_info.to_csv('/content/drive/MyDrive/Final Project Data Science Master/df_final.csv')